In [1]:
import datetime

def test():
    now = datetime.datetime.now()
    print("\njupyter cell run successfully. {}".format(now))

test()


jupyter cell run successfully. 2019-05-13 19:34:13.907183


In [3]:
import sys # For input/output to terminal
import os # For saving filepath

import matplotlib.pyplot as plt
%matplotlib inline

import requests # For requesting and downloading data set from URL
import gzip # For unzipping the data set
import pandas as pd # For handling dataframe
import numpy as np # For sparse grid


from scipy.sparse import csr_matrix # Sparse matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

test()


jupyter cell run successfully. 2019-05-13 19:34:29.225352


In [4]:
#################################################################################
#  Prepare the data set from Amazon
# Choose category here: https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt


# INSERT URL HERE
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
# url = "http://ctdbase.org/reports/CTD_genes_pathways.tsv.gz" # This is just an example data base which is smaller to handle

filename = url.split('/')[-1]
workingdir = os.path.abspath('')
datadir = "data" # Directory where data is stored
filepath_tsvgz = os.path.join(workingdir, datadir, filename)
filepath_tsv = filepath_tsvgz[0:-3]


while True:
    download_request = input("\nDo you want to prepare the following Amazon data set (can take a while if not downloaded): {}? [y]/[n] ".format(filename))
    print(filepath_tsvgz)


    if download_request == "y":

        def download_gz(url, filepath_tsvgz, filename):

            filepath_tsv = filepath_tsvgz[0:-3]

            # Check if data file already exists
            exists_tsvgz = os.path.isfile(filepath_tsvgz)
            if exists_tsvgz: print("\nFile already downloaded and compressed.")
            else:
                print("\nFile does not exist. Continue with download process.")

                print("\n{:^50}".format("...downloading the data..."))

                # Retrieve data from URL
                r = requests.get(url, stream=True)
                total_length = r.headers.get('content-length')

                with open(filepath_tsvgz, 'wb') as f:
                    # Show download progress bar
                    if total_length is None:
                        f.write(r.content)
                    else:
                        downloaded = 0
                        total_length = int(total_length)
                        for data in r.iter_content(chunk_size=max(int(total_length/1000), 1024*1024)):
                            downloaded += len(data)
                            f.write(data)
                            done = int(50*downloaded/total_length)
                            sys.stdout.write('\r[{}{}]'.format('█' * done, '.' * (50-done)))
                            sys.stdout.flush()

                print("Downloaded "+filename+" successfully.")

            # Unzip file  
            print("\n\nunzipping the file...")
            zipped = gzip.GzipFile(filepath_tsvgz, 'rb')
            zipped_content = zipped.read()
            zipped.close()
            unzipped = open(filepath_tsv, 'wb')
            unzipped.write(zipped_content)
            unzipped.close()
            print("\nFile decompressed successfully.")

            # Delete zipped file
            os.remove(filepath_tsvgz) # delete compressed data
            print("Compressed file deleted successfully.")
            print("File saved in the following location:\n{}".format(filepath_tsv))

        exists_tsv = os.path.isfile(filepath_tsv)
        # Execute download
        if not exists_tsv:
            download_gz(url, filepath_tsvgz, filename)
            break
        else: 
            print("\nFile already exists. Continue with creating data frame. ")
            break

    else:
        print("\nChecking if data is downloaded.")

        # Check if decompressed data file already exists
        exists = os.path.isfile(filepath_tsv)
        if exists:
            print("\nFile already exists. Continue with creating data frame.")
            break
        else:
            print("\nData file not found. Download the file to continue program.")
            
test()


Do you want to prepare the following Amazon data set (can take a while if not downloaded): amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz? [y]/[n]  n


C:\Users\azomm\Google Drive\Unil\FS19\Programming\git-final\program\data\amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz

Checking if data is downloaded.

File already exists. Continue with creating data frame.

jupyter cell run successfully. 2019-05-13 19:34:39.388459


In [5]:
###########################################
# Create dataframe



# Create pandas dataframe
print()
print("\n...creating dataframe...")
df = pd.read_csv(filepath_tsv, delimiter='\t', encoding="utf-8", error_bad_lines=False)

# Write dataframe to excel
# df = df.applymap(lambda x: x.encode('unicode_escape').
#              decode('utf-8') if isinstance(x, str) else x)
# df.to_excel("data.xlsx")

test()



...creating dataframe...


b'Skipping line 5782: expected 15 fields, saw 22\nSkipping line 31979: expected 15 fields, saw 22\nSkipping line 32408: expected 15 fields, saw 22\nSkipping line 45709: expected 15 fields, saw 22\nSkipping line 64585: expected 15 fields, saw 22\n'
b'Skipping line 115742: expected 15 fields, saw 22\nSkipping line 125537: expected 15 fields, saw 22\nSkipping line 127580: expected 15 fields, saw 22\nSkipping line 128696: expected 15 fields, saw 22\n'
b'Skipping line 143851: expected 15 fields, saw 22\nSkipping line 147069: expected 15 fields, saw 22\nSkipping line 149233: expected 15 fields, saw 22\nSkipping line 167536: expected 15 fields, saw 22\nSkipping line 168535: expected 15 fields, saw 22\nSkipping line 171659: expected 15 fields, saw 22\nSkipping line 184533: expected 15 fields, saw 22\nSkipping line 184624: expected 15 fields, saw 22\nSkipping line 184908: expected 15 fields, saw 22\n'
b'Skipping line 219150: expected 15 fields, saw 22\nSkipping line 223178: expected 15 fields, 


jupyter cell run successfully. 2019-05-13 19:34:59.601348


In [276]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,45610553,RMDCHWD0Y5OZ9,B00HH62VB6,618218723,AGPtek® 10 Isolated Output 9V 12V 18V Guitar P...,Musical Instruments,3,0,1,N,N,Three Stars,"Works very good, but induces ALOT of noise.",2015-08-31
1,US,14640079,RZSL0BALIYUNU,B003LRN53I,986692292,Sennheiser HD203 Closed-Back DJ Headphones,Musical Instruments,5,0,0,N,Y,Five Stars,Nice headphones at a reasonable price.,2015-08-31
2,US,6111003,RIZR67JKUDBI0,B0006VMBHI,603261968,AudioQuest LP record clean brush,Musical Instruments,3,0,1,N,Y,Three Stars,removes dust. does not clean,2015-08-31
3,US,1546619,R27HL570VNL85F,B002B55TRG,575084461,Hohner Inc. 560BX-BF Special Twenty Harmonica,Musical Instruments,5,0,0,N,Y,I purchase these for a friend in return for pl...,I purchase these for a friend in return for pl...,2015-08-31
4,US,12222213,R34EBU9QDWJ1GD,B00N1YPXW2,165236328,Blue Yeti USB Microphone - Blackout Edition,Musical Instruments,5,0,0,N,Y,Five Stars,This is an awesome mic!,2015-08-31


In [6]:
#####################################
# Nearest neighbors method
# (1) Create sparse matrix with unique customer and unique item index and the corresponding star rating
#    needed data: start_rating, product_id, customer_id
# (2) Compute nearest neighbors

# Calculate total number of products and customers (for range calculation)
prodNo = len(set(df["product_id"])) # Number of products
custNo = len(set(df["customer_id"])) # Number of customers
print("Number of unique products: {}".format(prodNo))
print("Number of unique customers: {}".format(custNo))

# Get unique product and customer IDs and index them
prodUnique_indexed = dict(zip(np.unique(df["product_id"]), list(range(prodNo))))
custUnique_indexed = dict(zip(np.unique(df["customer_id"]), list(range(custNo))))
prodUnique_reverseIndexed = dict(zip(list(range(prodNo)), np.unique(df["product_id"])))
custUnique_reverseIndexed = dict(zip(list(range(custNo)), np.unique(df["customer_id"])))

# Finally go through each row of dataframe and assign index 
custIndex = [custUnique_indexed[i] for i in df["customer_id"]]
prodIndex = [prodUnique_indexed[i] for i in df["product_id"]]

# Create indexed matrix
df_custIndex = pd.DataFrame(custIndex, columns=["custIndex"])
df_prodIndex = pd.DataFrame(prodIndex, columns=["prodIndex"])
df_indexed = pd.DataFrame(df["star_rating"], columns=["star_rating"])
df_indexed.insert(0, "prodIndex", df_prodIndex)
df_indexed.insert(0, "custIndex", df_custIndex)


# Create the sparse matrix based on a matrix with dimensions 
# (no. of unique products x no. of unique customres)
# Description from official scipy documentation:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].

df_csr = csr_matrix((df_indexed["star_rating"], (prodIndex, custIndex)), shape=(prodNo, custNo))

test()

Number of unique products: 123284
Number of unique customers: 572785

jupyter cell run successfully. 2019-05-13 19:35:10.849739


In [10]:
df[0:100]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,45610553,RMDCHWD0Y5OZ9,B00HH62VB6,618218723,AGPtek® 10 Isolated Output 9V 12V 18V Guitar P...,Musical Instruments,3,0,1,N,N,Three Stars,"Works very good, but induces ALOT of noise.",2015-08-31
1,US,14640079,RZSL0BALIYUNU,B003LRN53I,986692292,Sennheiser HD203 Closed-Back DJ Headphones,Musical Instruments,5,0,0,N,Y,Five Stars,Nice headphones at a reasonable price.,2015-08-31
2,US,6111003,RIZR67JKUDBI0,B0006VMBHI,603261968,AudioQuest LP record clean brush,Musical Instruments,3,0,1,N,Y,Three Stars,removes dust. does not clean,2015-08-31
3,US,1546619,R27HL570VNL85F,B002B55TRG,575084461,Hohner Inc. 560BX-BF Special Twenty Harmonica,Musical Instruments,5,0,0,N,Y,I purchase these for a friend in return for pl...,I purchase these for a friend in return for pl...,2015-08-31
4,US,12222213,R34EBU9QDWJ1GD,B00N1YPXW2,165236328,Blue Yeti USB Microphone - Blackout Edition,Musical Instruments,5,0,0,N,Y,Five Stars,This is an awesome mic!,2015-08-31
5,US,46018513,R1WCUI4Z1SIQEO,B001N4GRGS,134151483,Middle Atlantic Products QFAN-119,Musical Instruments,5,0,0,N,N,Five Stars,Used to cool equipment inside credenzas. Work...,2015-08-31
6,US,10225065,RL5LNO26GAVJ1,B009PJRMHQ,694166585,Kmise 1pc Pickguard for Gibson Sg Standard 3-p...,Musical Instruments,2,3,4,N,Y,Will not Fit Epiphone SG Special or any other ...,Note- Does not Fit Epiphone SG Special! Mine i...,2015-08-31
7,US,6356995,R3GYQ5W8JHP8SB,B00NKBDAZS,446431775,Kealoha Concert Ukulele - Stunning NEW Printed...,Musical Instruments,5,0,0,N,Y,Five Stars,Well built Ukulele! My daughter loves it!,2015-08-31
8,US,35297198,R30SHYQXGG5EYC,B006MIU7U2,125871705,Halco 80000 - MR16/3WW/FL/LED2 MR16 Flood LED ...,Musical Instruments,5,0,0,N,Y,Works fine. Hope it lasts through the warranty...,Had to replace a new light after a lightning s...,2015-08-31
9,US,32139520,R14YLXA56NP51I,B000FIBD0I,771888534,Gator GPTBLACK Plywood Pedal Board with Black ...,Musical Instruments,5,1,1,N,N,I upgraded the power cord to a heavier gauge w...,I've owned multiple fixed boards over the year...,2015-08-31


In [7]:
# Most reviews per product
numReview = pd.DataFrame(np.array(df_csr.getnnz(1)), columns=["numReview"]) # Gives array with number of review per product
numReview_indexed = pd.DataFrame(np.arange(custNo), columns=["prodIndex"])
numReview_indexed.insert(1, "numReview", numReview)
df_mostReviews = numReview_indexed.sort_values(by=["numReview"], ascending=False)[0:10]

a = np.array([])
for i in df_mostReviews["prodIndex"]:
    ii = prodUnique_reverseIndexed[i]
    a = np.append(a, ii)
df_a = pd.DataFrame(a, columns=["prod_id"])
df_mostReviews["prodIndex"] = df_a["prod_id"].values
print(df_mostReviews)


test()

        prodIndex  numReview
52324  B003VWJ2K8     3155.0
24008  B000ULAP4U     2917.0
50288  B003LPTAYI     2554.0
512    B00007E7C8     2451.0
14691  B000AJIF4E     2216.0
52325  B003VWKPHC     2144.0
44160  B002VA464S     2083.0
248    B000065BPB     2026.0
62233  B004XNK7AI     1985.0
29549  B0018TIADQ     1945.0

jupyter cell run successfully. 2019-05-13 19:35:13.318372


In [8]:
# Now we look at an example product "prod1"
prod1 = "B003VWJ2K8"
prod1_index = prodUnique_indexed[prod1]

# From the sparse matrix we extract all rows that are related to prod1.
# E.g. we look at all rows where a customer at least (!) reviewed B003VWJ2K8 
prod1_csr = df_csr[prod1_index]

amazon_url = "https://www.amazon.com/dp/"

NN_model = NearestNeighbors()
NN_model.fit(df_csr)
KNN = NN_model.kneighbors(prod1_csr, 10)

for i in KNN[1][0]:
    prod_id = prodUnique_reverseIndexed[i]
    print(amazon_url+prod_id)
    
test()

https://www.amazon.com/dp/B003VWJ2K8
https://www.amazon.com/dp/B000OR4A64
https://www.amazon.com/dp/B00AYPNC4C
https://www.amazon.com/dp/B009SPV5JI
https://www.amazon.com/dp/B0002GYX50
https://www.amazon.com/dp/B009NN1ISS
https://www.amazon.com/dp/B001JGHV8I
https://www.amazon.com/dp/B001RNHEAS
https://www.amazon.com/dp/B001JB6YD6
https://www.amazon.com/dp/B0002E3N30

jupyter cell run successfully. 2019-05-13 19:35:21.181463


In [326]:
# Now we do the same thing with another distance measure
# Default is euclidean
# But they suggest 

NN_model2 = NearestNeighbors(metric="cosine")
NN_model2.fit(df_csr)
KNN2 = NN_model2.kneighbors(prod1_csr, 10)

for i in KNN2[1][0]:
    prod_id = prodUnique_reverseIndexed[i]
    print(amazon_url+prod_id)
  
test()

https://www.amazon.com/dp/B003VWJ2K8
https://www.amazon.com/dp/B001PGXHXA
https://www.amazon.com/dp/B0002E1G5C
https://www.amazon.com/dp/B0006NDF8A
https://www.amazon.com/dp/B0009G1E0K
https://www.amazon.com/dp/B000OR2RNM
https://www.amazon.com/dp/B0002OOMU8
https://www.amazon.com/dp/B0002CZVWI
https://www.amazon.com/dp/B000OR4A64
https://www.amazon.com/dp/B00AYPNC4C

jupyter cell run successfully. 2019-05-09 20:45:54.121529


In [ ]:
# Sperate between train and test data
# 20% test data 
df_custIndex = pd.DataFrame(custIndex, columns=["custIndex"])
df_prodIndex = pd.DataFrame(prodIndex, columns=["prodIndex"])
y = pd.DataFrame(df["star_rating"], columns=["star_rating"])
X = df_prodIndex
X.insert(0, "custIndex", df_custIndex)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) 

# Put X, y back together
X_train.insert(2, "star_rating", y_train)
X_test.insert(2, "star_rating", y_test)

# Handle sparse matrix
#df_train = csr_matrix((X_train["star_rating"], (prodIndex, custIndex)), shape=(prodNo, custNo))

test()